In [1]:
import os
import sys
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
df = pd.read_csv('/Users/laiyuchun/Desktop/all_20200302.csv', encoding='big5')
cwd = os.getcwd()
ColNames = df.columns
DataFrameWithoutNa = df.dropna()
DataFrameResetIndex = DataFrameWithoutNa.reset_index(drop=True)


In [3]:
import numpy as np
NewCols = []
for i in range(1, 5):
    ColNames = ['染料_' + str(i), '濃度_' + str(i)]
    NewCols.append(ColNames)

NewCols = np.asarray(NewCols).reshape(8).tolist()


In [4]:
SplitSolubilityString = []
for i in range(len(DataFrameResetIndex['Drug Solubility'])):
    SplitSolubilityString.append(DataFrameResetIndex['Drug Solubility'][i].split(' '))
SpiteStringWithComma = []
for i in range(len(SplitSolubilityString)):
    FixLengthOfSolubility = []
    for j in range(len(SplitSolubilityString[i])):
        SpitedString = SplitSolubilityString[i][j].split(',')
        FixLengthOfSolubility.append(SpitedString)
    SpiteStringWithComma.append(FixLengthOfSolubility)


In [5]:
from functools import reduce
MergeSolubilityList = []
for i in range(len(SpiteStringWithComma)):
    MergeFragmentList = reduce(lambda list_1, list_2: list_1 + list_2, SpiteStringWithComma[i])
    ListWithoutNone = list(filter(None, MergeFragmentList))
    MergeSolubilityList.append(ListWithoutNone)

In [6]:
CleanOutput = []
LogAbnormalData = []
for i in MergeSolubilityList:
    if len(i) > 8:
        LogAbnormalData.append(i)

    else:
        CleanOutput.append(i)

In [7]:
TrainingData = pd.read_excel('/Users/laiyuchun/Desktop/hong_make_all_revised.xlsx', encoding='big5')

'''
this scope is to complement value because solubility requires
four kinds of dyestuff and concentration of four dyestuff. That
is to say there are totally eight columns. These columns would
be merged with NewCols to form a new dataframe
'''

# TODO It should remove abnormal solubility from DataFrameResetIndex in below step
ComplementaryValue = []
for i in CleanOutput:
    if len(i) < 6:
        i = i + [" ", " ", " ", " "]
        ComplementaryValue.append(i)
    else:
        i = i + [" ", " "]
        ComplementaryValue.append(i)

In [8]:
# 單色光: light 布號: Device
DfSolubility = pd.DataFrame(ComplementaryValue, columns=NewCols)
LightDeviceGroup = TrainingData.groupby(['單色光', '布號']).groups
DeviceToLight = dict((v, k) for k, v in LightDeviceGroup)
DataFrameResetIndex['布號'].map(DeviceToLight)
DataFrameResetIndex['單色光'] = DataFrameResetIndex['布號'].map(DeviceToLight)
DataFrameResetIndex.to_excel('/Users/laiyuchun/Desktop/df_modify_2.xlsx')

In [51]:
DfConcat = DataFrameResetIndex.join(DfSolubility, how='outer')
